In [1]:
from qiskit.circuit import QuantumCircuit
from qiskit.circuit.library import PhaseEstimation, U1Gate
from pennylane import numpy as np
import pennylane as qml
from qiskit.compiler import transpile
from qiskit_aer import AerSimulator

In [2]:
def qiskit_circuit(theta, ancilla):
    u = U1Gate(2 * np.pi * theta)
    qc = QuantumCircuit(ancilla + 1, ancilla)
    # possible states for qubit at last index
    # |1> and -|1> which are the eigenstates of U at eigenvalue e^{2*pi*i*theta}
    qc.x(ancilla)  # must be an eigenstate of Unitary matrix U
    # qc.z(ancilla)
    pe = PhaseEstimation(ancilla, unitary=u)
    qc.compose(pe, qubits=list(range(ancilla + 1)), inplace=True)
    return qc

In [3]:
ancilla = 6
dev = qml.device("default.qubit", wires=ancilla + 1)

In [6]:
theta = 1 / 3

simulator = AerSimulator()
qc = qiskit_circuit(theta, ancilla)
isa_circuit = transpile(qc, simulator)
pl_func = qml.from_qiskit(isa_circuit)


@qml.qnode(dev)
def circuit():
    pl_func()
    return qml.probs(wires=range(ancilla))


probabilties = circuit()
outcome = np.argmax(probabilties)
print(f"Highest probability outcome: {bin(outcome)[2:]}")
estimated_phase = outcome / 2**ancilla
print(f"Estimated phase: {estimated_phase}")

Highest probability outcome: 10101
Estimated phase: 0.328125
